In [ ]:
#CSV文件的建立和写入，读取。
import csv
def write_csv(file_name,row_image):
    path = str(file_name)+'.csv'
    with open(path,'a',newline='') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(row_image)

def read_csv(file_name):
    path = str(file_name)+'.csv'
    with open(path,"r") as f:
        csv_read = csv.reader(f)
        for line in csv_read:
            print(line)
def write_csv_rows(file_name,row_image):
    path = str(file_name)+'.csv'
    with open(path,'a',newline='') as f:
        csv_write = csv.writer(f)
        csv_write.writerows(row_image)

In [8]:
object_image_message= []
image_message= []
train_filename_list = []

In [9]:
# 将获取的图片数据集进行文件名称获取和保存。
import os
import random
import argparse
from os import getcwd
parser = argparse.ArgumentParser()
parser.add_argument('--jpeg_path',default = 'test_data',type = str, help = 'input test_jpeg label path')
parser.add_argument('--xml_path',default = 'train_data/Annotations',type = str, help = 'input xml label path')
parser.add_argument('--txt_path',default = './Images/main',type = str, help = 'output txt label path')
parser.add_argument('--test_path',default = './',type = str, help = 'output test_jpeg path')
opt = parser.parse_args(args=[])
abs_path = os.getcwd()
trainval_percent = 1.0
train_percent = 1.0
xmlfilepath = opt.xml_path
txtsavepath = opt.txt_path
testsavepath = opt.test_path
total_xml = os.listdir(xmlfilepath)
jpegfilepath = opt.jpeg_path
total_jpeg = os.listdir(jpegfilepath)
if not os.path.exists(txtsavepath):
	os.makedirs(txtsavepath)

num = len(total_xml)
list_index = range(num)
tv = int(num*trainval_percent)
tr = int(num*train_percent)
trainval = random.sample(list_index,tv) # 查一下这个sample
train = random.sample(trainval,tr)

file_trainval = open(txtsavepath + '/trainval.txt','w')
file_test = open(testsavepath + '/test.txt','w')
file_train = open(txtsavepath + '/train.txt','w')
file_val = open(txtsavepath + '/val.txt','w')
number=0
for i in list_index:
	name = total_xml[i][:-4]+'\n'
	if i in trainval:
		file_trainval.write(name)
		train_filename_list.append(total_xml[i][:-4]+'.jpg')
		if i in train:
			file_train.write(name)
		else:
			file_val.write(name)
		if number%3==0:
			file_val.write(name)
		number+=1
for i in range(len(total_jpeg)):
	name = total_jpeg[i]+'\n'
	file_test.write( '/hy-tmp/test_data/'+name) 
file_trainval.close()
file_test.close()
file_train.close()
file_val.close()

In [15]:
# 对xml数据进行处理，提取关键的数据。
import xml.etree.ElementTree as ET
import os
from os import getcwd

sets = ['train','val']
classes = ['paper_trash','packed_trash','snakeskin_bag','plastic_trash','carton','stone_waste','sand_waste','foam_trash','metal_waste','wood_waste']# 改成自己的类别
#abs_path = os.getcwd()
abs_path = '/hy-tmp'
# print(abs_path)
#'./hy-tmp/train_data/new_images/%s.jpg
def convert(size,box):
	dw = 1. /(size[0])
	dh = 1. / (size[1])
	x = (box[0]+box[1])/2.0 -1
	y = (box[2]+box[3])/2.0 -1
	w = box[1] - box[0]
	h = box[3] - box[2]
	x = x*dw
	w = w*dw
	y = y*dh
	h = h*dh
	return x,y,w,h
def convert_annotation(image_id):
	#print(image_id)
	#classes = ['paper_trash','packed_trash','snakeskin_bag','plastic_trash','carton','stone_waste', 'sand_waste', 'foam_trash','metal_waste','wood_waste']
	in_file = open('./train_data/Annotations/%s.xml'%(image_id),encoding='utf-8')
	out_file = open('./labels/%s.txt'%(image_id),'w')
	tree = ET.parse(in_file)
	root = tree.getroot()
	size = root.find('size')
	w = int(size.find('width').text)
	h = int(size.find('height').text)
	image_message.append([image_id,w,h,w*h])
	for obj in root.iter('object'):
		if obj.find('difficult'):
			difficult = obj.find('difficult').text
		else:
			difficult = 0
		cls = obj.find('name').text
		if cls not in classes or int(difficult)==1:
			continue
		cls_id = classes.index(cls)
		xmlbox = obj.find('bndbox')
		b = (float(xmlbox.find('xmin').text),float(xmlbox.find('xmax').text),float(xmlbox.find('ymin').text),float(xmlbox.find('ymax').text))
		b1,b2,b3,b4 = b
		object_image_message.append([image_id,w,h,w*h,cls_id,b1,b2,b3,b4,(b2-b1)*(b4-b3)])
		#标注越界修正
		if b2>w:
			b2 =w
		if b4>h:
			b4=h
		b = (b1,b2,b3,b4)
		bb = convert((w,h),b)
		if (b2-b1)*(b4-b3)>0:# 存在object框选范围面积为0的情况
			out_file.write(str(cls_id)+' '+' '.join([str(a) for a in bb])+'\n')

wd = getcwd()
for image_set in sets:
	if not os.path.exists('./labels/'):
		os.makedirs('./labels/')
	image_ids = open('./Images/main/%s.txt'%(image_set)).read().split('\n')[:-1]
	list_file = open('%s.txt'%(image_set),'w')
	for image_id in image_ids:
		list_file.write(abs_path + '/train_data/images/%s.jpg\n'%(image_id))
		#print(image_id)
		convert_annotation(image_id)
	#list_file.close()


In [ ]:
image_message

In [ ]:
object_image_message

In [ ]:
len(object_image_message)

In [ ]:
len(image_message)

In [ ]:
for i in object_image_message:
    if i[-1]<=0:# object是否有0像素的框图
        print('area_object is 0')
        print(i[4])# cls_id
        print(i[0])#image_name
        print(i)
    if (i[-2]>i[2])or(i[-4]>i[1])or(i[-3]<0)or(i[-5]<0):#判断object是否有框选区域超出边界的情况。
        print('the object is in question')
        print(i[0])
        print(i)
    if i[-2]<i[-3]:
        print(i)
    if i[-4]<i[-5]:
        print(i)
# 所有有一个 名称是 A10351的cls_id为4的object有问题，应该剔除或者调整一下。   还有不少object框选区域超出边界的情况

In [ ]:
# 对上述情况进行处理
new_object_image_message = object_image_message.copy()

for i in new_object_image_message:
    if i[-1]<=0:# object是否有0像素的框图
        print(i)
        new_object_image_message.pop(new_object_image_message.index(i))
        #print(new_object_image_message.index(i))
        print('1')
    if i[-2]>i[2]:
        i[-2]=i[2]
        print('2')
    if i[-4]>i[1]:
        i[-4]=i[1]
        print('3')
    if i[-3]<0:
        i[-3]=0
        print('4')
    if i[-5]<0:
        i[-5]=0
        print('5')

In [ ]:
# 把想要的信息写入csv文件
header_object_image_message = ['image_name','width','height','image_area','cls_id','xmin','xmax','ymin','ymax','object_area']
header_image_message = ['image_name','width','height','image_area']
write_csv('image_message',header_image_message)
write_csv_rows('image_message',image_message)
write_csv('object_image_message',header_object_image_message)
write_csv_rows('object_image_message',object_image_message)

In [ ]:
# 对数据进行处理后写入新的文档
write_csv('new_object_image_message',header_object_image_message)
write_csv_rows('new_object_image_message',new_object_image_message)

In [2]:
import pandas as pd
import matplotlib as plt
df = pd.read_csv('./new_object_image_message.csv')

In [ ]:
df.head()

参考url：http://c.biancheng.net/pandas/plot.html

In [ ]:
df[['image_area','object_area']].plot(bins=30,kind = 'hist')

In [ ]:
df.index.values

In [ ]:
df['id'] = df.index
df.plot.scatter(y='image_area',x='id')

In [ ]:
df['id'] = df.index
df.plot.scatter(y='object_area',x='id')

In [ ]:
df[['image_area','object_area']].plot.scatter(x='image_area',y='object_area')

In [ ]:
df.head()

In [ ]:
df.info()

In [3]:
# 多裁剪一些数据。我想把那些object图片都裁剪下来做一些训练
# url：https://blog.csdn.net/qq_43622870/article/details/123970530
from cv2 import cv2
import os

def convert(size,box):
	dw = 1. /(size[0])
	dh = 1. / (size[1])
	x = (box[0]+box[1])/2.0 -1
	y = (box[2]+box[3])/2.0 -1
	w = box[1] - box[0]
	h = box[3] - box[2]
	x = x*dw
	w = w*dw
	y = y*dh
	h = h*dh
	return x,y,w,h

def split_image(src_path, x_start, x_end, y_start, y_end, save_path,cls_id):
    # for file in file_names:
    # src_path 具体图片路径，包含后缀
    img = cv2.imread(src_path)
    # cv2.imwrite(path, img)
    size = img.shape[0:2]
    w = size[1]
    h = size[0]
    # print(file, w, h)
    # 每行的高度和每列的宽度
    num = 0
	# 保存切割好的图片的路径，记得要填上后缀，以及名字要处理一下，可以是
	# src_path.split('.')[0] + '_' + str((i+1)*(j+1)) + '.jpg'
    row_start = int((x_start-0)//100*90)
    row_end = int((w - x_end)//100*10+x_end)
    col_start = int((y_start-0)//100*90)
    col_end = int((h-y_end)//100*10+y_end)
    #row_start = int(x_start)
    #row_end = int(x_end)
    #col_start = int(y_start)
    #col_end = int(y_end)
    # print(row_start, row_end, col_start, col_end)
    # cv2图片： [高， 宽]
    child_img = img[col_start:col_end,row_start:row_end]
    cv2.imwrite(save_path, child_img)
    new_objects =((x_start-0)/100*10,x_end-row_start,(y_start-0)/100*10,y_end-col_start)  #正常的坐标值，做一下归一化处理
    new_x,new_y,new_w,new_h = convert((row_end-row_start,col_end-col_start),new_objects)   # 归一化后的值
    result = [cls_id,new_x,new_y,new_w,new_h]
    return result
#return (x_start-0)/5*2 ,x_end-row_start,(y_start-0)/5*2,y_end-col_start # 返回的是新的object框的值（xmin,xmax,ymin,ymax）


In [18]:
abs_path = os.getcwd()
print(abs_path)
list_file = open('%s.txt'%('train'),'a+')
for i in range(len(df['image_name'])):
    if 1000<df['object_area'][i]<3000:
        src_path = abs_path+'/train_data/images/'+df['image_name'][i]+'.jpg'
        save_path = abs_path+'/train_data/new_images/'+df['image_name'][i]+ '_' +str(df['cls_id'][i])+ '_' +str(i)+'.jpg'
        #print(save_path)
        list_file.write('/hy-tmp/train_data/new_images/%s.jpg\n'%(df['image_name'][i]+ '_' +str(df['cls_id'][i])+ '_' +str(i)))
        new_objects = split_image(src_path, df['xmin'][i], df['xmax'][i], df['ymin'][i], df['ymax'][i], save_path,df['cls_id'][i]) # 归一化后的新的object的值
        f = open('./train_data/new_images_labels/'+df['image_name'][i]+ '_' +str(df['cls_id'][i])+ '_' +str(i)+'.txt','w')
        f.write(' '.join([str(a) for a in new_objects])+'\n')
        f.close()
list_file.close()

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210513174622169_0_3_1.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210514080813988_0_3_4.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210514080813988_0_0_6.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210514123428815_0_3_9.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210516080706443_0_5_11.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210516080706443_0_5_12.jpg
E:\Python from jupyter file\Urban Govern

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210814132159015_0_2_3057.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210814132159015_0_2_3059.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210814132159015_0_2_3062.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210814132159015_0_2_3063.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210814132159015_0_2_3065.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210814132356714_0_2_3068.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210815142402987_0_4_4216.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210815142402987_0_4_4217.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210815142634330_0_0_4223.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210815142704335_0_0_4226.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210815142832106_0_1_4234.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210815142832106_0_1_4235.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210908183750642_0_4_5625.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210908183750642_0_4_5626.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210908183750642_0_4_5627.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210908183750642_0_4_5630.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210910063501237_0_2_5643.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20210910063501237_0_2_5644.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20220128123005400_0_4_6955.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20220128123005400_0_4_6956.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20220128123005400_0_4_6957.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20220128123005400_0_4_6958.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20220128123005400_0_4_6959.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20220128123005400_0_4_6960.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/20

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A (9733)_4_9115.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A (996)_4_9134.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A (999)_3_9142.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A (999)_3_9143.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A (999)_3_9144.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A (999)_3_9145.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A (999)_3_9146.jpg
E:\Python from jupyter file\Urban Governance Garbage E

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-10351_4_9781.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-10351_2_9783.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-10352_2_9786.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-10353_4_9788.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-10353_4_9789.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-10353_4_9790.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-10353_2_9791.jpg
E:\Python from jupyter file\Urban Governance Garbage Ex

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-32_4_11527.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-3204_0_11537.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-3381_2_11563.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-3381_4_11564.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-3382_4_11565.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-3382_2_11566.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-341_0_11568.jpg
E:\Python from jupyter file\Urban Governance Garbage Expos

E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-9706_0_14147.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-9710_0_14149.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-9711_3_14153.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-9712_0_14155.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-9712_3_14156.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-9713_3_14158.jpg
E:\Python from jupyter file\Urban Governance Garbage Exposure Detection Algorithm Competition/train_data/new_images/A-9713_0_14159.jpg
E:\Python from jupyter file\Urban Governance Garbage Ex

In [ ]:
os.getcwd()

In [ ]:
a = [1,2,3,4]
a.appendleft(4)